In [ ]:
import requests
import time
import re
from lxml import etree #lxml: 解析网页，用 xpath 表达式与正则表达式一起来获取网页信息，相比 bs4 速度更快。
from io import open

import time

file_name = ""

def get_areas(url):
    print('start grabing areas')
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.108 Safari/537.36'}
    resposne = requests.get(url, headers=headers)  # 获得网页的原始 html
    content = etree.HTML(resposne.text)
    #print etree.tostring(content)    
    
    # Get Regions
    areas = content.xpath("//dd[@data-index = '0']//div[@class='option-list']/a//text()")
    print ("-----------------------")
    print ("抓取到的区域有")
    for x in areas:
        print x,
    print 
    print ("总共有" + str(len(areas)) +"个区域")
    print ("-----------------------")
    
    # Get Related Region Links
    areas_link = content.xpath("//dd[@data-index = '0']//div[@class='option-list']/a/@href")
    
    # 不统计最后两个区域
    for i in range(1,len(areas)-2):
        area = areas[i]
        area_link = areas_link[i]
        link = 'https://bj.lianjia.com' + area_link
        print link
        print("开始抓取页面")
        get_pages(area, link)
        
#通过获取某一区域的页数，来拼接某一页的链接
def get_pages(area,area_link):
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.108 Safari/537.36'}
    resposne = requests.get(area_link, headers=headers)
    #print resposne.text
    find_total_pages =  re.findall("page-data=\'{\"totalPage\":(\d+),\"curPage\"", resposne.text)
    if (find_total_pages == []):
        print ("该区域无房源信息")
        return 
    pages =  int(find_total_pages[0])
    print("这个区域有" + str(pages) + "页")
    for page in range(1, pages+1):
    #for page in range(1, 2):
        #url = 'https://bj.lianjia.com/zufang/dongcheng/pg' + str(page)
        url = area_link + 'pg' + str(page)
        print("开始抓取" + str(page) + r'/' + str(pages) +"的信息")
        print url
        get_house_info(area,url)



        
#获取某一区域某一页的详细房租信息
def get_house_info(area, url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.108 Safari/537.36'}
    time.sleep(2)
    try:
        
        resposne = requests.get(url, headers=headers)
        content = etree.HTML(resposne.text)
        info=[]
        
        
        total_house_index_number = int(re.findall("data-index=\"(\d+)\"", resposne.text)[-1])
        #print total_house_index_number
        houselist = content.xpath('//ul[@id="house-lst"]//li/div/a/@href')
        
        for i in range(total_house_index_number+1):        
            title = content.xpath("//div[@class='where']/a/span/text()")[i]
            room_type = content.xpath("//div[@class='where']/span[1]/span/text()")[i]
            square = re.findall("(\d+)",content.xpath("//div[@class='where']/span[2]/text()")[i])[0]
            position = content.xpath("//div[@class='where']/span[3]/text()")[i].replace(" ", "")
            
            #print title
            #print room_type
            #print square
            #print position
            
            try:
                detail_place = re.findall("([\u4E00-\u9FA5]+)租房", content.xpath("//div[@class='other']/div/a/text()")[i])[0]
            except Exception as e:
                detail_place = ""
            #print detail_place
            
            
            try:
                floor =re.findall("([\u4E00-\u9FA5]+)\(", content.xpath("//div[@class='other']/div/text()[1]")[i])[0]
            except Exception as e:
                floor = ""
            
            try:
                total_floor = re.findall("(\d+)",content.xpath("//div[@class='other']/div/text()[1]")[i])[0]            
            except Exception as e:
                total_floor = ""
            #print floor
            #print total_floor 
            
            
            try:
                house_year = re.findall("(\d+)",content.xpath("//div[@class='other']/div/text()[2]")[i])[0]
            except Exception as e:
                house_year = ""                
            
            price = content.xpath("//div[@class='col-3']/div/span/text()")[i]
            
            contents_lon_lat = requests.get(houselist[i])
            contents_lon_lat = etree.HTML(contents_lon_lat.content.decode('utf-8'))
            resblockPosition = contents_lon_lat.xpath('/html/body/script[12]/text()').pop()
            items = re.search("resblockPosition:\'(.+)\'", resblockPosition)
            items_group = items.group() # 经纬度
            longitude_latitude = items_group.split(':')[1]
            longitude_latitude = longitude_latitude.split('\'')[1]
            [lon, lat] = longitude_latitude.split(',')
            #print lon, lat

            with open(file_name,'a',encoding='utf-8') as f:
                f.write(area + ',' + title + ',' + room_type + ',' + square + ',' +position+
                       ','+ detail_place+','+floor+','+total_floor+','+price+','+house_year+','+lon +',' + lat +'\n')
                
            print i, area, title, i, "/", total_house_index_number, "done!"            
            #print('writing work has done!continue the next page')
            
    except Exception as e:
        print( 'ooops! connecting error, retrying.....')
        time.sleep(20)
        return get_house_info(area, url)

        
def main():
    print('start!')
    global file_name
    timestr = time.strftime("%Y%m%d_%H%M%S")
    file_name = "/home/ubuntu/01.workspace/03.crawer_rent_price/" + timestr + "-data_list.txt"
    with open(file_name,'w',encoding='utf-8') as f:
        print (file_name) 
        
    url = 'https://bj.lianjia.com/zufang'    
    get_areas(url)
    print ("Finish!")
    
if __name__ == '__main__':
    main()

start!
/home/ubuntu/01.workspace/03.crawer_rent_price/20180826_202948-data_list.txt
start grabing areas
-----------------------
抓取到的区域有
不限 东城 西城 朝阳 海淀 丰台 石景山 通州 昌平 大兴 亦庄开发区 顺义 房山 门头沟 平谷 怀柔 密云 延庆 燕郊 香河
总共有20个区域
-----------------------
https://bj.lianjia.com/zufang/dongcheng/
开始抓取页面
这个区域有19页
开始抓取1/19的信息
https://bj.lianjia.com/zufang/dongcheng/pg1
0 东城 和平新城一期  0 / 29 done!
1 东城 胡家园小区  1 / 29 done!
2 东城 丽水湾畔家园  2 / 29 done!
3 东城 上龙西里  3 / 29 done!
4 东城 西革新里110号院  4 / 29 done!
5 东城 幸福家园一期  5 / 29 done!
6 东城 富贵园四区  6 / 29 done!
7 东城 富贵园一区  7 / 29 done!
8 东城 幸福家园一期  8 / 29 done!
9 东城 安外花园  9 / 29 done!
10 东城 金鱼池中区  10 / 29 done!
11 东城 望陶园小区  11 / 29 done!
12 东城 六铺炕  12 / 29 done!
13 东城 金桥国际  13 / 29 done!
14 东城 安德路47号院  14 / 29 done!
15 东城 安德路47号院  15 / 29 done!
16 东城 和平新城一期  16 / 29 done!
17 东城 金桥国际  17 / 29 done!
18 东城 青年沟  18 / 29 done!
19 东城 管村26号院  19 / 29 done!
20 东城 工体北里  20 / 29 done!
21 东城 安定路  21 / 29 done!
22 东城 和平里五区  22 / 29 done!
23 东城 新永外文化用品商城  23 / 29 done!
24 东城 和平新城一期  24 /

9 东城 富贵园三区  9 / 29 done!
10 东城 当代MOMA  10 / 29 done!
11 东城 金桥国际  11 / 29 done!
12 东城 宇飞大厦  12 / 29 done!
13 东城 雍和家园二期  13 / 29 done!
14 东城 朝阳首府  14 / 29 done!
15 东城 禾风仕家  15 / 29 done!
16 东城 东总布胡同19号院  16 / 29 done!
17 东城 盛德大厦  17 / 29 done!
18 东城 望陶园小区  18 / 29 done!
19 东城 当代MOMA  19 / 29 done!
20 东城 自由季  20 / 29 done!
21 东城 贡院九号  21 / 29 done!
22 东城 大羊宜宾胡同33号院  22 / 29 done!
23 东城 太华公寓  23 / 29 done!
24 东城 盛德大厦  24 / 29 done!
25 东城 北官厅胡同2号院  25 / 29 done!
26 东城 NAGA上院  26 / 29 done!
27 东城 烧酒胡同3号院  27 / 29 done!
28 东城 正东国际大厦  28 / 29 done!
29 东城 前门前  29 / 29 done!
开始抓取11/19的信息
https://bj.lianjia.com/zufang/dongcheng/pg11
0 东城 前门前  0 / 29 done!
1 东城 凯景铭座  1 / 29 done!
2 东城 左安漪园  2 / 29 done!
3 东城 建国门北大街5号  3 / 29 done!
4 东城 使馆壹号院  4 / 29 done!
5 东城 雅安国际公寓  5 / 29 done!
6 东城 新奥洋房  6 / 29 done!
7 东城 当代MOMA  7 / 29 done!
8 东城 中海紫御公馆  8 / 29 done!
9 东城 聚龙花园  9 / 29 done!
10 东城 东直门内北小街2号院  10 / 29 done!
11 东城 海运仓胡同3号院  11 / 29 done!
12 东城 和平里一区  12 / 29 done!
13 东城 东直门内北小街8号院  13 / 29 done!

6 西城 北广家园  6 / 29 done!
7 西城 北营房西里  7 / 29 done!
8 西城 爱民里小区  8 / 29 done!
9 西城 爱民里小区  9 / 29 done!
10 西城 乐城  10 / 29 done!
11 西城 百万庄辰区  11 / 29 done!
12 西城 扣钟北里  12 / 29 done!
13 西城 三里河一区  13 / 29 done!
14 西城 红莲南里  14 / 29 done!
15 西城 侨办大院  15 / 29 done!
16 西城 北礼士路62号院  16 / 29 done!
17 西城 三里河三区  17 / 29 done!
18 西城 虎坊路小区  18 / 29 done!
19 西城 六铺炕三区  19 / 29 done!
20 西城 牛街西里  20 / 29 done!
21 西城 白菜湾社区  21 / 29 done!
22 西城 西黄城根45号院  22 / 29 done!
23 西城 小红庙  23 / 29 done!
24 西城 水电部宿舍  24 / 29 done!
25 西城 白菜湾四巷17号院  25 / 29 done!
26 西城 南纬路2号院  26 / 29 done!
27 西城 椿树园  27 / 29 done!
28 西城 南线阁5号院  28 / 29 done!
29 西城 天宁寺东里  29 / 29 done!
开始抓取2/32的信息
https://bj.lianjia.com/zufang/xicheng/pg2
0 西城 新外大街甲28号  0 / 29 done!
1 西城 新外大街甲8号院  1 / 29 done!
2 西城 新外大街甲8号院  2 / 29 done!
3 西城 爱民里小区  3 / 29 done!
4 西城 考拉社区  4 / 29 done!
5 西城 红莲南里  5 / 29 done!
6 西城 佟麟阁路62号院  6 / 29 done!
7 西城 姚家井二巷  7 / 29 done!
8 西城 马甸南村  8 / 29 done!
9 西城 天宁寺前街北里  9 / 29 done!
10 西城 新外大街甲8号院  10 / 29 done!
11 西城 黑窑厂东街  11

In [84]:
import requests
import time
import re
from lxml import etree #lxml: 解析网页，用 xpath 表达式与正则表达式一起来获取网页信息，相比 bs4 速度更快。
from io import open
import pandas as pd
import time

filename = "/home/ubuntu/01.workspace/03.crawer_rent_price/20180826_194154-data_list.txt"
#filename = "/home/ubuntu/01.workspace/03.crawer_rent_price/untitled.txt"
with open(filename, 'r') as f:
    print ("Sucess Open the File")

df = pd.read_csv(filename, header=None, encoding='utf-8', names = ['areas', 'title', 'room_type', 'square', 'position', 'detail_place', 'floor', 'total_floor', 'price', 'house_year', 'lon', 'lat'])
#df = pd.read_csv(filename, names = ['areas', 'title', 'room_type'])
df

Sucess Open the File


,areas,title,room_type,square,position,detail_place,floor,total_floor,price,house_year,lon,lat
0,东城,和平新城一期,4室1厅,17,东,NaN,NaN,19,4530,2003.0,116.432835,39.957774
1,东城,安外大街,1室1厅,46,南,NaN,NaN,16,6460,1994.0,116.414549,39.972390
2,东城,西革新里108号院,2室1厅,59,南,NaN,NaN,7,6930,1995.0,116.398033,39.872301
3,东城,十字坡东里,1室1厅,38,南,NaN,NaN,5,6690,1986.0,116.445818,39.945717
4,东城,金桥国际,1室1厅,54,北,NaN,NaN,21,7460,2004.0,116.448876,39.901694
5,东城,安德路47号院,1室1厅,36,东,NaN,NaN,6,6360,1990.0,116.407868,39.958020
6,东城,安德路47号院,1室1厅,36,东,NaN,NaN,6,6590,1990.0,116.407868,39.958020
7,东城,六铺炕,3室1厅,11,南,NaN,NaN,6,3430,1980.0,116.400533,39.962226
8,东城,望陶园小区,2室1厅,78,北,NaN,NaN,18,7290,2003.0,116.400737,39.870957
9,东城,金鱼池中区,3室1厅,10,南,NaN,NaN,6,2760,2002.0,116.415431,39.896044


In [88]:
print (df.describe())

           square  detail_place  floor  total_floor         price  \
count  390.000000           0.0    0.0   390.000000    390.000000   
mean    41.405128           NaN    NaN    11.979487   4557.948718   
std     45.609948           NaN    NaN     7.592080   3340.553099   
min      6.000000           NaN    NaN     1.000000   1330.000000   
25%     11.000000           NaN    NaN     6.000000   2407.500000   
50%     18.000000           NaN    NaN     8.000000   3745.000000   
75%     59.000000           NaN    NaN    18.000000   5797.500000   
max    374.000000           NaN    NaN    32.000000  30000.000000   

        house_year         lon         lat  
count   368.000000  390.000000  390.000000  
mean   2000.157609  116.382351   39.921550  
std      10.402114    0.161319    0.116009  
min    1958.000000  116.079970   39.688632  
25%    1994.000000  116.258665   39.847719  
50%    2002.000000  116.376144   39.920301  
75%    2007.250000  116.487565   39.975678  
max    2017.000000

In [87]:
#北京路段_房屋均价分布图
detail_place = df.groupby(['detail_place'])
house_com = detail_place['price'].agg(['mean','count'])
house_com.reset_index(inplace=True)
detail_place_main = house_com.sort_values('count',ascending=False)[0:20]

attr = detail_place_main['detail_place']
v1 = detail_place_main['count']
v2 = detail_place_main['mean']

line = Line("北京主要路段房租均价")
line.add("路段",attr,v2,is_stack=True,xaxis_rotate=30,yaxix_min=4.2,
mark_point=['min','max'],xaxis_interval=0,line_color='lightblue',    line_width=4,mark_point_textcolor='black',mark_point_color='lightblue',
    is_splitline_show=False)

bar = Bar("北京主要路段房屋数量")
bar.add("路段",attr,v1,is_stack=True,xaxis_rotate=30,yaxix_min=4.2,
    xaxis_interval=0,is_splitline_show=False)

overlap = Overlap()
overlap.add(bar)
overlap.add(line,yaxis_index=1,is_add_yaxis=True)
overlap.render('北京路段_房屋均价分布图.html')

NameError: name 'Line' is not defined